In [2]:
#HTTP를 처리하기 위한 라이브러리
import urllib.request
from urllib.request import urlretrieve

#requests 라이브러리 기반으로 코드 작성하지 말것!!!
#import requests
import time
from time import sleep
import os
import shutil
from datetime import datetime, timedelta

#HTML을 처리하기 위한 라이브러리
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
dstUrl = r'https://news.naver.com/main/history/mainnews/list.nhn?date=2018-08-10'
webpage = urllib.request.urlopen(dstUrl)
# ori_contents = webpage.read().decode("utf-8")
ori_contents = webpage.read()
contents = BeautifulSoup(ori_contents, "html.parser")
contents

<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="http://news.naver.com/main/home.nhn" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공" property="og:description"/>
<meta content="summary" name="twitter:card"/>
<meta content="네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="twitter:image"/>
<meta content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공" name="twitter:description"/>
<title>주요뉴스 : 

In [2]:
dstUrl = r'https://news.naver.com/main/history/mainnews/text.nhn?date=2018-08-10&page=7'
webpage = urllib.request.urlopen(dstUrl)
# ori_contents = webpage.read().decode("utf-8")
ori_contents = webpage.read()
contents = BeautifulSoup(ori_contents, "html.parser")
contents

<ul class="mlist2">
<li>
<a class="nclicks(his_lst.text,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=100&amp;oid=001&amp;aid=0010261824" target="_blank">합수단, 조현천 자진귀국 설득 총력…여권무효화 일단 유보</a>
<span class="mlist2_info">
<span class="writing">연합뉴스</span>
<span class="v_bar"></span>
<span class="eh_edittime">08/10 06:12 ~ 08/10 08:09 (1시간57분) 노출</span>
</span>
</li>
<li>
<a class="nclicks(his_lst.text,,2)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=102&amp;oid=003&amp;aid=0008749977" target="_blank">김경수, 출석 20시간만에 귀가…"특검이 답 내놓을 차례"</a>
<span class="mlist2_info">
<span class="writing">뉴시스</span>
<span class="v_bar"></span>
<span class="eh_edittime">08/10 06:03 ~ 08/10 09:17 (3시간14분) 노출</span>
</span>
</li>
<li>
<a class="nclicks(his_lst.text,,3)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=102&amp;oid=001&amp;aid=0010261805" target="_blank">'우산 챙기세요' 곳곳 강한 소나기…최고 35도 폭염은 계속</a>
<span class="mlist2_info

In [7]:
# 뉴스URL(URL)
contents('li')[0]('a')[0]['href']

'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0010261824'

In [8]:
# 뉴스제목(Title)
contents('li')[0]('a')[0].text

'합수단, 조현천 자진귀국 설득 총력…여권무효화 일단 유보'

In [9]:
# 뉴스매체(Media)
contents('li')[0]('span')[1].text

'연합뉴스'

In [11]:
# 노출(Exposure)
contents('li')[0]('span')[3].text

'08/10 06:12 ~ 08/10 08:09 (1시간57분) 노출'

In [13]:
dstUrl = r'https://news.naver.com/main/history/mainnews/text.nhn?date=2009-06-07&page=7'
webpage = urllib.request.urlopen(dstUrl)
# ori_contents = webpage.read().decode("utf-8")
ori_contents = webpage.read()
contents = BeautifulSoup(ori_contents, "html.parser")
contents('li')[0]('a')[0]['href']
contents('li')[0]('a')[0].text
contents('li')[0]('span')[1].text
try:
    contents('li')[0]('span')[3].text
except:
    print("노출 정보가 없습니다.")

노출 정보가 없습니다.


https://news.naver.com/main/history/mainnews/list.nhn?date=2018-08-10  
https://news.naver.com/main/history/mainnews/photoTv.nhn?date=2018-08-10&page=2  
https://news.naver.com/main/history/mainnews/text.nhn?date=2018-08-10&page=7  
- 그 다음 페이지와 동일하면 마지막임
- 2009년 6월 7일부터

In [7]:
def naverMainnewsText(startDate, endDate):
    columnList = ("date", "title", "url", "media", "exposure")
    mainnewsText = pd.DataFrame(columns=columnList)
    mainnewsText
    startDate = datetime.strptime(startDate, "%Y-%m-%d").date()
    endDate = datetime.strptime(endDate, "%Y-%m-%d").date()
    
    while(startDate <= endDate):
#         print(startDate, "작업을 시작합니다.")
        mainnewsPage = 1
        checkHref = ''
        
        for mainnewsPage in range(1, 15):
#             print(mainnewsPage, "페이지입니다.")
            naverMainnewsUrl = r"https://news.naver.com/main/history/mainnews/text.nhn?date="
            naverMainnewsUrl = naverMainnewsUrl + str(startDate) + r"&page=" + str(mainnewsPage)
#             print(naverMainnewsUrl)
            
            webpage = urllib.request.urlopen(naverMainnewsUrl)
#             ori_contents = webpage.read().decode("utf-8")
            ori_contents = webpage.read()
            contents = BeautifulSoup(ori_contents, "html.parser")
#             print(contents)
            
#             print(contents('li')('a')[0]['href'])
            if checkHref == contents('li')[0]('a')[0]['href']: break
#             print(mainnewsPage, "페이지입니다.")
        
            for news in contents('li'):
                try:
                    newsExposure = news('span')[3].text
                except:
                    newsExposure = ""
                    
                newsInfo = [startDate, news('a')[0].text, news('a')[0]['href'], news('span')[1].text, newsExposure]
#                 len(newsInfo)
                mainnewsText = mainnewsText.append(pd.DataFrame(data=[newsInfo], columns=columnList))
#                 mainnewsPhototv = pd.concat(mainnewsPhototv, pd.DataFrame(newsInfo, columns=columnList))
#                 title, url, media, exposure, image
#                 print([startDate, news('a')[0]['title'], news('a')[0]['href'], news('span')[1].text, news('span')[2].text, news('img')[0]['src']])
#                 mainnewsPhototv = mainnewsPhototv.append(pd.DataFrame([str(startDate), news('a')[0]['title'], 
#                                                                        news('a')[0]['href'], news('span')[1].text, 
#                                                                        news('span')[2].text, news('img')[0]['src']], columns=columnList))
    
            checkHref = contents('li')[0]('a')[0]['href']
#         urlretrieve(bigkindsUrl, str(startDate) + '.xls')
        
        mainnewsText.to_excel(str(startDate) + r".xls", index=False)
        print(startDate, "처리되었습니다.")
        startDate = startDate + timedelta(days = 1)
        sleep(1)

In [8]:
startDate = '2009-06-07'
endDate = '2019-06-07'

naverMainnewsText(startDate, endDate)
# naverMainnewsPhototv(startDate, endDate)

2009-06-07 처리되었습니다.
2009-06-08 처리되었습니다.
2009-06-09 처리되었습니다.
2009-06-10 처리되었습니다.
2009-06-11 처리되었습니다.
2009-06-12 처리되었습니다.
2009-06-13 처리되었습니다.


KeyboardInterrupt: 

In [80]:
mainnewsPhototv

NameError: name 'mainnewsPhototv' is not defined

In [136]:
dstUrl = r'https://news.naver.com/main/history/mainnews/photoTv.nhn?date=2009-06-07&page=2'
webpage = urllib.request.urlopen(dstUrl)
# ori_contents = webpage.read().decode("utf-8")
ori_contents = webpage.read()
contents = BeautifulSoup(ori_contents, "html.parser")
contents


<ul class="edit_history_lst">
<li>
<a class="nclicks(his_lst.img,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=001&amp;aid=0002702261" target="_blank" title="관세청 `희망의 운동화' 캄보디아에 기증">
<img alt="관세청 `희망의 운동화' 캄보디아에 기증" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=102;" src="https://imgnews.pstatic.net/image/upload/item20090607/6689_untitled-12_copy.jpg" width="150"/>
<em class="eh_tit">관세청 `희망의 운동화' 캄보디아에 기증</em>
<span class="trans_border"></span>
</a>
<span class="eh_by">연합뉴스</span>
</li>
<li>
<a class="nclicks(his_lst.img,,2)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=106&amp;oid=081&amp;aid=0002018614" target="_blank" title="딸 졸업식 모인 브루스-데미무어 전 커플">
<img alt="딸 졸업식 모인 브루스-데미무어 전 커플" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;

In [137]:
contents('li')

[<li>
 <a class="nclicks(his_lst.img,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=001&amp;aid=0002702261" target="_blank" title="관세청 `희망의 운동화' 캄보디아에 기증">
 <img alt="관세청 `희망의 운동화' 캄보디아에 기증" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=102;" src="https://imgnews.pstatic.net/image/upload/item20090607/6689_untitled-12_copy.jpg" width="150"/>
 <em class="eh_tit">관세청 `희망의 운동화' 캄보디아에 기증</em>
 <span class="trans_border"></span>
 </a>
 <span class="eh_by">연합뉴스</span>
 </li>, <li>
 <a class="nclicks(his_lst.img,,2)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=106&amp;oid=081&amp;aid=0002018614" target="_blank" title="딸 졸업식 모인 브루스-데미무어 전 커플">
 <img alt="딸 졸업식 모인 브루스-데미무어 전 커플" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=10

In [138]:
contents('li')[0]

<li>
<a class="nclicks(his_lst.img,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=001&amp;aid=0002702261" target="_blank" title="관세청 `희망의 운동화' 캄보디아에 기증">
<img alt="관세청 `희망의 운동화' 캄보디아에 기증" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=102;" src="https://imgnews.pstatic.net/image/upload/item20090607/6689_untitled-12_copy.jpg" width="150"/>
<em class="eh_tit">관세청 `희망의 운동화' 캄보디아에 기증</em>
<span class="trans_border"></span>
</a>
<span class="eh_by">연합뉴스</span>
</li>

In [139]:
contents('li')[0]('a')

[<a class="nclicks(his_lst.img,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=001&amp;aid=0002702261" target="_blank" title="관세청 `희망의 운동화' 캄보디아에 기증">
 <img alt="관세청 `희망의 운동화' 캄보디아에 기증" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=102;" src="https://imgnews.pstatic.net/image/upload/item20090607/6689_untitled-12_copy.jpg" width="150"/>
 <em class="eh_tit">관세청 `희망의 운동화' 캄보디아에 기증</em>
 <span class="trans_border"></span>
 </a>]

In [140]:
# 기사URL
contents('li')[0]('a')[0]['href']

'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=001&aid=0002702261'

In [141]:
# 기사제목
contents('li')[0]('a')[0]['title']

"관세청 `희망의 운동화' 캄보디아에 기증"

In [142]:
contents('li')[0]('span')[1].text

'연합뉴스'

In [144]:
if (contents('li')[0]('span')[2].text) == False: ""

IndexError: list index out of range

In [34]:
contents('li')[0]('img')[0]['src']

'https://imgnews.pstatic.net/image/upload/item/2018/08/10/213405986_1.jpg?type=f270_166'

In [35]:
for news in contents('li'):
#     title, url, media, exposure
    print([news('a')[0]['title'], news('a')[0]['href'], news('span')[1].text, news('span')[2].text], news('img')[0]['src'])

['"대머리가 어때서"…가발 벗고 단체 커플 댄스', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=449&aid=0000156811', '채널A', '08/10 21:53 ~ 08/10 23:44 (1시간51분) 노출'] https://imgnews.pstatic.net/image/upload/item/2018/08/10/213405986_1.jpg?type=f270_166
['광주·전남 호우특보 확대…최고 84mm 비에 광주 도로 침수', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=001&aid=0010263806', '연합뉴스', '08/10 21:19 ~ 08/10 22:35 (1시간15분) 노출'] https://imgnews.pstatic.net/image/upload/item/2018/08/10/211905655_1.jpg?type=f270_166
['"개는 가축 아닙니다" 검토…개 식용 금지될까', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=214&aid=0000865012', 'MBC 뉴스', '08/10 21:09 ~ 08/11 05:51 (8시간42분) 노출'] https://imgnews.pstatic.net/image/upload/item/2018/08/10/210906561_1.jpg?type=f270_166
["작전명 '태양을 만진다'…코로나 비밀 캐러 우주로", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=022&aid=0003295685', '세계일보', '08/10 20:04 ~ 08/10 21:19 (1시간14분) 노출'] https://imgnews.pstatic.net/image/upload/item/2018/08/10

In [15]:
contents('li')[0]('em')[0].text

'"대머리가 어때서"…가발 벗고 단체 커플 댄스'

In [16]:
contents('li')[0]('strong')[0].text

'동영상기사'

In [8]:
contents.find("ul", attrs={'class': 'edit_history_lst'})

<ul class="edit_history_lst">
<li>
<a class="nclicks(his_lst.img,,1)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=102&amp;oid=214&amp;aid=0000865018" target="_blank" title="'등교 거부' 학생들…&quot;석면 공포에 등교 못해요&quot;">
<img alt="'등교 거부' 학생들…&quot;석면 공포에 등교 못해요&quot;" height="102" onerror="javascript:this.src='https://imgnews.pstatic.net/image/news/2009/noimage_150x102.gif';this.style.width=150;this.style.height=102;" src="https://imgnews.pstatic.net/image/upload/item/2018/08/10/230306523_1.jpg?type=f270_166" width="150"/>
<em class="eh_tit">'등교 거부' 학생들…"석면 공포에 등교 못해요"</em>
<strong class="r_ico r_vod_small">동영상기사</strong>
<span class="trans_border"></span>
</a>
<span class="eh_by">MBC 뉴스</span>
<span class="eh_time">08/10 23:44 ~ 08/11 06:05 (6시간20분) 노출</span>
</li>
<li>
<a class="nclicks(his_lst.img,,2)" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=shm&amp;sid1=102&amp;oid=448&amp;aid=0000249609" target="_blank" title='일가족 3명 화재로 의식불명…소방당국 "에어컨 과열 